In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
model = keras.models.load_model("policy_model")

In [3]:
import os 
import io
import numpy as np
import chess.pgn
from leela_board import LeelaBoard

fens = []
with open(os.path.join("pgns", "century.pgn")) as f:
    game = chess.pgn.read_game(f)


all_planes = []
indices = [2, 5, 16]
board = LeelaBoard()
for i, move in enumerate(game.mainline_moves()):
    board.pc_board.push(move)
    board._lcz_push()
    if i in indices:
        fen = board.pc_board.fen()
        planes = board.lcz_features()
        fens.append(fen)
        all_planes.append(planes)

all_planes = np.array(all_planes)


In [4]:
# Adapted from https://keras.io/examples/vision/grad_cam/

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is not None:
            class_channel = preds[:, pred_index]
        else:
            # over everything
            class_channel = preds[:, :]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 2, 3))
    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = np.copy(last_conv_layer_output.numpy())
    for i in range(32):
        heatmap[i] *= pooled_grads[i]
    heatmap = heatmap.sum(axis=0)
    heatmap = np.squeeze(heatmap)
    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap

In [ ]:
# Change i in [0, 1, 2] to get the three results

from manim_chess.chess_board import ChessBoard
from manim import *
class HeatmapGrad(MovingCameraScene):
    def construct(self):
        i = 3

        heatmap = make_gradcam_heatmap(np.expand_dims(all_planes[i], 0), model, "policy_conv")
        turn = "w" in fens[i]
        if turn:
            heatmap = np.flipud(heatmap)
        main_board = ChessBoard(fens[i]).move_to(ORIGIN)
        main_board.set_piece_opacities_square(heatmap)

        if not turn:
            main_board.flip()
        self.add(main_board)

%manim -qh -v WARNING HeatmapGrad
